In [2]:
import configparser
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2 import connect
import numpy
from psycopg2.extras import execute_values
from datetime import datetime
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import copy
import rick
import geopandas as gpd
import seaborn as sns
import shapely
import matplotlib.patches as patches
import matplotlib.ticker as ticker
from setuptools import setup, find_packages
from math import factorial
from simanneal import Annealer
from itertools import islice 
import matplotlib.patches as patches
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
from itertools import combinations, chain
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [22]:
a = 0
month = ['congestion.metrics_v5_201901','congestion.metrics_v5_201902', 'congestion.metrics_v5_201903', 'congestion.metrics_v5_201904', 'congestion.metrics_v5_201905', 'congestion.metrics_v5_201906', 'congestion.metrics_v5_201907', 'congestion.metrics_v5_201908', 'congestion.metrics_v5_201909', 'congestion.metrics_v5_201910', 'congestion.metrics_v5_201911', 'congestion.metrics_v5_201912' ]
for i in month:
    a = a + 1
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''CREATE TABLE {0} AS
                    with highway as (select distinct segment_id from congestion.segment_links_v5 
                                             inner join (
                                             select routing_grid.* from congestion.routing_grid
                                             join here_gis.streets_att_18_3 on link_id =  (left(id::Text, -1))::numeric
                                             WHERE streets_att_19_4.st_name in ('LAKE SHORE BLVD E', 'LAKE SHORE BLVD W','DON VALLEY PKWY','HWY-404','HWY-401 COLLECTORS','GARDINER EXPY',
                                            'HWY-427','HWY-27', 'HWY-401 EXPRESS','HWY-400','HWY-2A', 'HWY-409', 'HWY-427 COLLECTORS') 
                                            or st_name ~~* 'Allen RD'::text and func_class != '4'
                                            )a using (link_dir))
                    , tti_prep as  (
                    SELECT a.segment_id,
                    a.datetime_bin::time AS time_bin,
                    COUNT(datetime_bin) AS num_bins,
                    b.seg_length,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    AVG(a.segment_tt_med_hc_corr) AS med_tt_corr,
                    case when highway.segment_id is not null then b.tt_baseline_10pct_corr else b.tt_baseline_25pct_corr end AS baseline_tt_corr,
                    b.tt_baseline_25pct_med_corr AS baseline_tt_med_corr

                    FROM (select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} ) a
                    LEFT JOIN congestion.tt_segments_baseline_v5_2019_af b USING (segment_id)
                    left join highway using (segment_id)	

                    WHERE a.segment_tt_avg_all IS NOT NULL
                    GROUP BY segment_id, highway.segment_id, datetime_bin::time, b.seg_length, tt_baseline_10pct_corr, tt_baseline_25pct_corr, tt_baseline_25pct_med, tt_baseline_25pct_med_corr
                    ORDER BY segment_id, datetime_bin::time
                    ) 
                    , bi_prep_am as (
                    SELECT a.segment_id,
                    '08:00:00'::time without time zone AS time_bin,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    PERCENTILE_CONT (0.95) WITHIN GROUP (ORDER BY a.segment_tt_avg_hc_corr ASC) AS pct95_tt_corr, 
                    count(segment_tt_avg_hc_corr) as num_bins

                    FROM (
                    select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} and datetime_bin::time <@ '[08:00:00, 08:30:00]'::timerange) a
                    group by segment_id )

                    , bi_prep_pm as (
                    SELECT a.segment_id,
                    '17:00:00'::time without time zone AS time_bin,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    PERCENTILE_CONT (0.95) WITHIN GROUP (ORDER BY a.segment_tt_avg_hc_corr ASC) AS pct95_tt_corr, 
                    count(segment_tt_avg_hc_corr) as num_bins

                    FROM (
                    select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} and datetime_bin::time <@ '[17:00:00, 17:30:00]'::timerange) a
                    group by segment_id )	


                    SELECT segment_id, time_bin, tti.num_bins as tti_num_bins,
                    tti.avg_tt_corr, tti.med_tt_corr, tti.baseline_tt_corr, tti.baseline_tt_med_corr, 
                    tti.avg_tt_corr / tti.baseline_tt_corr AS tti_corr,
                    tti.med_tt_corr / tti.baseline_tt_med_corr AS tti_med_corr, 
                    COALESCE((am.pct95_tt_corr- am.avg_tt_corr)/am.avg_tt_corr,(pm.pct95_tt_corr- pm.avg_tt_corr)/pm.avg_tt_corr) as bi,
                    COALESCE(am.num_bins, pm.num_bins) as bi_num_bins

                    from tti_prep tti
                    left join bi_prep_am am using (segment_id, time_bin)
                    left join bi_prep_pm pm using (segment_id, time_bin)
'''.format(i,a)
            cur.execute(sql)

congestion.metrics_v5_201901
congestion.metrics_v5_201902
congestion.metrics_v5_201903
congestion.metrics_v5_201904
congestion.metrics_v5_201905
congestion.metrics_v5_201906
congestion.metrics_v5_201907
congestion.metrics_v5_201908
congestion.metrics_v5_201909
congestion.metrics_v5_201910
congestion.metrics_v5_201911
congestion.metrics_v5_201912


In [ ]:
a = 0
month = ['congestion.metrics_v5_202001_19_4','congestion.metrics_v5_202002_19_4', 'congestion.metrics_v5_202003_19_4', 
         'congestion.metrics_v5_202004_19_4', 'congestion.metrics_v5_202005_19_4', 'congestion.metrics_v5_202006_19_4', ]
for i in month:
    a = a + 1
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''CREATE TABLE {0} AS
                    with highway as (select distinct segment_id from congestion.segment_links_v5_19_4 
                                             inner join (
                                             select routing_grid.* from congestion.routing_grid
                                             join here_gis.streets_att_19_4 on link_id =  (left(id::Text, -1))::numeric
                                             WHERE streets_att_19_4.st_name::text = 'LAKE SHORE BLVD E'::text 
                                               OR streets_att_19_4.st_name::text = 'LAKE SHORE BLVD W'::text 
                                               OR streets_att_19_4.st_name::text = 'DON VALLEY PKWY'::text 
                                               OR streets_att_19_4.st_name::text = 'HWY-404'::text 
                                               OR streets_att_19_4.st_name::text ~~* '%40%'::text 
                                               OR streets_att_19_4.st_name::text ~~* '%427%'::text 
                                               OR streets_att_19_4.st_name::text ~~* '%27%'::text 
                                               OR streets_att_19_4.st_name::text = 'GARDINER EXPY'::text
                                               OR streets_att_19_4.st_name::text 
                                            )a using (link_dir))
                    , tti_prep as  (
                    SELECT a.segment_id,
                    a.datetime_bin::time AS time_bin,
                    COUNT(datetime_bin) AS num_bins,
                    b.seg_length,
                    AVG(a.segment_tt_avg_hc) AS avg_tt_corr,
                    case when highway.segment_id is not null then b.tt_baseline_10pct_corr else b.tt_baseline_25pct_corr end AS baseline_tt_corr,
                    b.tt_baseline_25pct_med_corr AS baseline_tt_med_corr

                    FROM (select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} ) a
                    LEFT JOIN congestion.tt_segments_baseline_v5_2019_af b USING (segment_id)
                    left join highway using (segment_id)	

                    WHERE a.segment_tt_avg_all IS NOT NULL
                    GROUP BY segment_id, highway.segment_id, datetime_bin::time, b.seg_length, tt_baseline_10pct_corr, tt_baseline_25pct_corr, tt_baseline_25pct_med, tt_baseline_25pct_med_corr
                    ORDER BY segment_id, datetime_bin::time
                    ) 
                    , bi_prep_am as (
                    SELECT a.segment_id,
                    '08:00:00'::time without time zone AS time_bin,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    PERCENTILE_CONT (0.95) WITHIN GROUP (ORDER BY a.segment_tt_avg_hc_corr ASC) AS pct95_tt_corr, 
                    count(segment_tt_avg_hc_corr) as num_bins

                    FROM (
                    select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} and datetime_bin::time <@ '[08:00:00, 08:30:00]'::timerange) a
                    group by segment_id )

                    , bi_prep_pm as (
                    SELECT a.segment_id,
                    '17:00:00'::time without time zone AS time_bin,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    PERCENTILE_CONT (0.95) WITHIN GROUP (ORDER BY a.segment_tt_avg_hc_corr ASC) AS pct95_tt_corr, 
                    count(segment_tt_avg_hc_corr) as num_bins

                    FROM (
                    select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} and datetime_bin::time <@ '[17:00:00, 17:30:00]'::timerange) a
                    group by segment_id )	


                    SELECT segment_id, time_bin, tti.num_bins as tti_num_bins,
                    tti.avg_tt_corr, tti.med_tt_corr, tti.baseline_tt_corr, tti.baseline_tt_med_corr, 
                    tti.avg_tt_corr / tti.baseline_tt_corr AS tti_corr,
                    tti.med_tt_corr / tti.baseline_tt_med_corr AS tti_med_corr, 
                    COALESCE((am.pct95_tt_corr- am.avg_tt_corr)/am.avg_tt_corr,(pm.pct95_tt_corr- pm.avg_tt_corr)/pm.avg_tt_corr) as bi,
                    COALESCE(am.num_bins, pm.num_bins) as bi_num_bins

                    from tti_prep tti
                    left join bi_prep_am am using (segment_id, time_bin)
                    left join bi_prep_pm pm using (segment_id, time_bin)
'''.format(i,a)
            cur.execute(sql)

In [26]:
# Gets quarter metrics 
with con:
    with con.cursor() as cur:
        sql = '''CREATE TABLE congestion.metrics_v5_2019_q1 AS
                    with highway as (select distinct segment_id from congestion.segment_links_v5 
                                             inner join (
                                             select routing_grid.* from congestion.routing_grid
                                             join here_gis.streets_att_18_3 on link_id =  (left(id::Text, -1))::numeric
                                             WHERE streets_att_18_3.st_name::text = 'LAKE SHORE BLVD E'::text OR streets_att_18_3.st_name::text = 'LAKE SHORE BLVD W'::text 
                                               OR streets_att_18_3.st_name::text = 'DON VALLEY PKWY'::text OR streets_att_18_3.st_name::text = 'HWY-404'::text 
                                               OR streets_att_18_3.st_name::text ~~* '%40%'::text OR streets_att_18_3.st_name::text ~~* '%427%'::text OR streets_att_18_3.st_name::text ~~* '%27%'::text 
                                               OR streets_att_18_3.st_name::text = 'GARDINER EXPY'::text
                                            )a using (link_dir))
                    , tti_prep as  (
                    SELECT a.segment_id,
                    a.datetime_bin::time AS time_bin,
                    COUNT(datetime_bin) AS num_bins,
                    b.seg_length,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    AVG(a.segment_tt_med_hc_corr) AS med_tt_corr,
                    case when highway.segment_id is not null then b.tt_baseline_10pct_corr else b.tt_baseline_25pct_corr end AS baseline_tt_corr,
                    b.tt_baseline_25pct_med_corr AS baseline_tt_med_corr

                    FROM (select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) in (1,2,3) ) a
                    LEFT JOIN congestion.tt_segments_baseline_v5_2019_af b USING (segment_id)
                    left join highway using (segment_id)	

                    WHERE a.segment_tt_avg_all IS NOT NULL
                    GROUP BY segment_id, highway.segment_id, datetime_bin::time, b.seg_length, tt_baseline_10pct_corr, tt_baseline_25pct_corr, tt_baseline_25pct_med, tt_baseline_25pct_med_corr
                    ORDER BY segment_id, datetime_bin::time
                    ) 
                    , bi_prep_am as (
                    SELECT a.segment_id,
                    '08:00:00'::time without time zone AS time_bin,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    PERCENTILE_CONT (0.95) WITHIN GROUP (ORDER BY a.segment_tt_avg_hc_corr ASC) AS pct95_tt_corr, 
                    count(segment_tt_avg_hc_corr) as num_bins

                    FROM (
                    select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) in (1,2,3) and datetime_bin::time <@ '[08:00:00, 08:30:00]'::timerange) a
                    group by segment_id )

                    , bi_prep_pm as (
                    SELECT a.segment_id,
                    '17:00:00'::time without time zone AS time_bin,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    PERCENTILE_CONT (0.95) WITHIN GROUP (ORDER BY a.segment_tt_avg_hc_corr ASC) AS pct95_tt_corr, 
                    count(segment_tt_avg_hc_corr) as num_bins

                    FROM (
                    select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) in (1,2,3) and datetime_bin::time <@ '[17:00:00, 17:30:00]'::timerange) a
                    group by segment_id )	


                    SELECT segment_id, time_bin, tti.num_bins as tti_num_bins,
                    tti.avg_tt_corr, tti.med_tt_corr, tti.baseline_tt_corr, tti.baseline_tt_med_corr, 
                    tti.avg_tt_corr / tti.baseline_tt_corr AS tti_corr,
                    tti.med_tt_corr / tti.baseline_tt_med_corr AS tti_med_corr, 
                    COALESCE((am.pct95_tt_corr- am.avg_tt_corr)/am.avg_tt_corr,(pm.pct95_tt_corr- pm.avg_tt_corr)/pm.avg_tt_corr) as bi,
                    COALESCE(am.num_bins, pm.num_bins) as bi_num_bins

                    from tti_prep tti
                    left join bi_prep_am am using (segment_id, time_bin)
                    left join bi_prep_pm pm using (segment_id, time_bin)
                '''
        cur.execute(sql)

In [20]:
month = ['here.ta_201901','here.ta_201902', 'here.ta_201903', 'here.ta_201904', 'here.ta_201905',
         'here.ta_201906', 'here.ta_201907', 'here.ta_201908', 'here.ta_201909','here.ta_201910', 
         'here.ta_201911', 'here.ta_201912']
for i in month:
    with con:
        with con.cursor() as cur:
            sql = '''INSERT INTO congestion.tt_segments_1hr_v5_2019
                        with speed_links as (
                    select segment_id, 
					link_dir,
					length AS link_length, 
					(datetime_bin(tx,60)) AS datetime_bin,
					harmean(mean_corr) AS spd_avg_hc_corr,
					COUNT (DISTINCT tx)  AS count_hc
					from {0}
                    inner join congestion.segment_links_v5 using (link_dir)
                    WHERE confidence >= 30
					GROUP BY segment_id, link_dir, datetime_bin, length)
--hc=high confidence >= 30
SELECT segment_id, datetime_bin, 
CASE WHEN SUM(link_length) >= 0.8 * b.length 
	THEN SUM(link_length / spd_avg_hc_corr  * 3.6 ) * b.length / SUM(link_length)
	END AS segment_tt_avg_hc_corr ,	
	SUM(link_length) / b.length * 100 AS data_pct_hc
FROM speed_links
INNER JOIN congestion.segments_v5 b 
USING (segment_id)
WHERE link_length / spd_avg_hc_corr  IS NOT NULL
GROUP BY segment_id, datetime_bin, b.length
ORDER BY segment_id, datetime_bin'''.format(i)
            cur.execute(sql)

In [9]:
month = ['here.ta_202002', 'here.ta_202003', 'here.ta_202004', 'here.ta_202005',
         'here.ta_202006']
for i in month:
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''INSERT INTO congestion.tt_segments_30mins_v5_2020_19_4
                    with speed_links as (
                    select segment_id, 
					link_dir,
					length AS link_length, 
					(datetime_bin(tx,30)) AS datetime_bin,
					harmean(mean) AS spd_avg_hc,
					COUNT (DISTINCT tx)  AS count_hc
					from {0}
                    inner join congestion.segment_links_v5_19_4 using (link_dir)
                    WHERE confidence >= 30
					GROUP BY segment_id, link_dir, datetime_bin, length)
                    SELECT segment_id, datetime_bin, 
                    CASE WHEN SUM(link_length) >= 0.8 * b.length 
                        THEN SUM(link_length / spd_avg_hc  * 3.6 ) * b.length / SUM(link_length)
                        END AS segment_tt_avg_hc ,	
                        SUM(link_length) / b.length * 100 AS data_pct_hc
                    FROM speed_links
                    INNER JOIN congestion.segments_v5 b 
                    USING (segment_id)
                    WHERE link_length / spd_avg_hc  IS NOT NULL
                    GROUP BY segment_id, datetime_bin, b.length
                    ORDER BY segment_id, datetime_bin'''.format(i)
            cur.execute(sql)

here.ta_202002
here.ta_202003
here.ta_202004
here.ta_202005
here.ta_202006


In [ ]:
month = ['here.ta_202002', 'here.ta_202003', 'here.ta_202004', 'here.ta_202005',
         'here.ta_202006']
for i in month:
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''INSERT INTO congestion.tt_segments_30_min_v5_2019_af
                    with speed_links as (
                    select segment_id, 
					link_dir,
					length AS link_length, 
					(datetime_bin(tx,30)) AS datetime_bin,
					harmean(mean) AS spd_avg_hc,
					COUNT (DISTINCT tx)  AS count_hc
					from {0}
                    inner join congestion.segment_links_v5_19_4 using (link_dir)
                    WHERE confidence >= 30
					GROUP BY segment_id, link_dir, datetime_bin, length)
                    
                    SELECT segment_id, datetime_bin, 
                    CASE WHEN SUM(link_length) >= 0.8 * b.length 
                        THEN SUM(link_length / spd_avg_hc  * 3.6 ) * b.length / SUM(link_length)
                        END AS segment_tt_avg_hc ,	
                        SUM(link_length) / b.length * 100 AS data_pct_hc
                    FROM speed_links
                    INNER JOIN congestion.segments_v5 b 
                    USING (segment_id)
                    WHERE link_length / spd_avg_hc  IS NOT NULL
                    GROUP BY segment_id, datetime_bin, b.length
                    ORDER BY segment_id, datetime_bin'''.format(i)
            cur.execute(sql)

In [14]:
#Speed links
month = ['here.ta_202001']
for i in month:
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''CREATE TABLE congestion.speeds_links_30_2020_v5_19_4 AS

                    with here_prep as (
                    select link_dir, length, tx, stddev, confidence, mean, pct_50 from 
                    {0}
                    inner join congestion.segment_links_v5_19_4 using (link_dir))
                    , here as (
                    select * from here_prep
                    LEFT JOIN ref.holiday hol ON hol.dt = here_prep.tx::date
                    where hol.dt IS NULL AND date_part('isodow'::text, tx::date)::integer < 6)	

                    SELECT X.segment_id, X.link_dir, X.length AS link_length, X.datetime_bin, 
                    X.spd_avg_all, Y.spd_avg_hc, 
                    X.spd_med_all, Y.spd_med_hc,
                    X.count_all, Y.count_hc
                    FROM
                    (
                    SELECT a.segment_id, a.link_dir, 
                    (datetime_bin(b.tx,30)) AS datetime_bin,
                    harmean(mean) AS spd_avg_all,
                    harmean(pct_50) AS spd_med_all, 			  
                    COUNT (DISTINCT b.tx) AS count_all, b.length
                    FROM congestion.segment_links_v5_19_4 a
                    INNER JOIN here b
                    USING (link_dir)
                    GROUP BY a.segment_id, a.link_dir, datetime_bin, b.length
                    ) X

                    LEFT JOIN

                    (
                    SELECT a.segment_id, a.link_dir, 
                    (datetime_bin(b.tx,30)) AS datetime_bin,
                    harmean(mean) AS spd_avg_hc,
                    harmean(pct_50) AS spd_med_hc,		  
                    COUNT (DISTINCT b.tx)  AS count_hc, b.length
                    FROM congestion.segment_links_v5_19_4 a
                    INNER JOIN here b
                    USING (link_dir)
                    WHERE confidence >= 30
                    GROUP BY a.segment_id, a.link_dir, datetime_bin, b.length
                        )  Y

                    USING (segment_id, link_dir, datetime_bin, length)
                    ORDER BY segment_id, link_dir, datetime_bin'''.format(i)
            cur.execute(sql)

here.ta_202001


In [6]:
#Speed links
month = ['here.ta_201901','here.ta_201902', 'here.ta_201903', 'here.ta_201904', 'here.ta_201905', 'here.ta_201906', 'here.ta_201907', 'here.ta_201908', 'here.ta_201909', 'here.ta_201910', 'here.ta_201911', 'here.ta_201912' ]
for i in month:
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''insert into  congestion.speeds_links_30_2019_v5_tbdeleted

                     with here_prep as (
                    select link_dir, length, tx, stddev, confidence, mean, pct_50, mean_corr, pct_50_corr from 
                    {0}_old
                    inner join congestion.segment_links_v5 using (link_dir)
                    where segment_id in (22282,22283))
                    , here as (
                    select * from here_prep
                    LEFT JOIN ref.holiday hol ON hol.dt = here_prep.tx::date
                    where hol.dt IS NULL AND date_part('isodow'::text, tx::date)::integer < 6)	

                    SELECT X.segment_id, X.link_dir, X.length AS link_length, X.datetime_bin, 
                    X.spd_avg_all, X.spd_avg_all_corr, Y.spd_avg_hc, Y.spd_avg_hc_corr, 
                    X.spd_med_all, X.spd_med_all_corr, Y.spd_med_hc, Y.spd_med_hc_corr,
                    X.count_all, Y.count_hc
                    FROM
                    (
                    SELECT a.segment_id, a.link_dir, 
                    (datetime_bin(b.tx,30)) AS datetime_bin,
                    harmean(mean) AS spd_avg_all,
                    harmean(pct_50) AS spd_med_all,
                    harmean(mean_corr) AS spd_avg_all_corr,
                    harmean(pct_50_corr) AS spd_med_all_corr, 			  
                    COUNT (DISTINCT b.tx) AS count_all, b.length
                    FROM congestion.segment_links_v5 a
                    INNER JOIN here b
                    USING (link_dir)
                    GROUP BY a.segment_id, a.link_dir, datetime_bin, b.length
                    ) X

                    LEFT JOIN

                    (
                    SELECT a.segment_id, a.link_dir, 
                    (datetime_bin(b.tx,30)) AS datetime_bin,
                    harmean(mean) AS spd_avg_hc,
                    harmean(pct_50) AS spd_med_hc,
                    harmean(mean_corr) AS spd_avg_hc_corr,
                    harmean(pct_50_corr) AS spd_med_hc_corr, 			  
                    COUNT (DISTINCT b.tx)  AS count_hc, b.length
                    FROM congestion.segment_links_v5 a
                    INNER JOIN here b
                    USING (link_dir)
                    WHERE confidence >= 30
                    GROUP BY a.segment_id, a.link_dir, datetime_bin, b.length
                        )  Y

                    USING (segment_id, link_dir, datetime_bin, length)
                    ORDER BY segment_id, link_dir, datetime_bin'''.format(i)
            cur.execute(sql)

here.ta_201901
here.ta_201902
here.ta_201903
here.ta_201904
here.ta_201905
here.ta_201906
here.ta_201907
here.ta_201908
here.ta_201909
here.ta_201910
here.ta_201911
here.ta_201912


In [7]:
#Speed links
month = ['here.ta_202007']
for i in month:
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''INSERT INTO congestion.speeds_links_30_2020_v5_19_4 

                    with here_prep as (
                    select link_dir, length, tx, stddev, confidence, mean, pct_50 from 
                    {0}
                    inner join congestion.segment_links_v5_19_4 using (link_dir)
                    )
                    , here as (
                    select * from here_prep
                    LEFT JOIN ref.holiday hol ON hol.dt = here_prep.tx::date
                    where hol.dt IS NULL AND date_part('isodow'::text, tx::date)::integer < 6)	

                    SELECT X.segment_id, X.link_dir, X.length AS link_length, X.datetime_bin, 
                    X.spd_avg_all, Y.spd_avg_hc, 
                    X.spd_med_all, Y.spd_med_hc,
                    X.count_all, Y.count_hc
                    FROM
                    (
                    SELECT a.segment_id, a.link_dir, 
                    (datetime_bin(b.tx,30)) AS datetime_bin,
                    harmean(mean) AS spd_avg_all,
                    harmean(pct_50) AS spd_med_all, 			  
                    COUNT (DISTINCT b.tx) AS count_all, b.length
                    FROM congestion.segment_links_v5_19_4 a
                    INNER JOIN here b
                    USING (link_dir)
                    GROUP BY a.segment_id, a.link_dir, datetime_bin, b.length
                    ) X

                    LEFT JOIN

                    (
                    SELECT a.segment_id, a.link_dir, 
                    (datetime_bin(b.tx,30)) AS datetime_bin,
                    harmean(mean) AS spd_avg_hc,
                    harmean(pct_50) AS spd_med_hc,		  
                    COUNT (DISTINCT b.tx)  AS count_hc, b.length
                    FROM congestion.segment_links_v5_19_4 a
                    INNER JOIN here b
                    USING (link_dir)
                    WHERE confidence >= 30
                    GROUP BY a.segment_id, a.link_dir, datetime_bin, b.length
                        )  Y

                    USING (segment_id, link_dir, datetime_bin, length)
                    ORDER BY segment_id, link_dir, datetime_bin'''.format(i)
            cur.execute(sql)

here.ta_202007
